## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-17-19-11-49 +0000,nyt,GQ Names Adam Baidawi as Its Top Editor,https://www.nytimes.com/2026/02/17/business/me...
1,2026-02-17-19-11-06 +0000,nypost,NY GOP governor contender Bruce Blakeman rips ...,https://nypost.com/2026/02/17/us-news/bruce-bl...
2,2026-02-17-19-10-51 +0000,nypost,Viral photo of ‘sexy’ LA cop sends hearts raci...,https://nypost.com/2026/02/17/us-news/viral-ph...
3,2026-02-17-19-09-59 +0000,nypost,Mayor Bass doubles down on comments about LA28...,https://nypost.com/2026/02/17/us-news/karen-ba...
4,2026-02-17-19-08-57 +0000,bbc,Shein under EU investigation over childlike se...,https://www.bbc.com/news/articles/cr7321n1n0eo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2463/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
62,trump,32
86,jackson,19
85,jesse,16
81,talks,13
70,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
195,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...,91
72,2026-02-17-17-15-17 +0000,nypost,Trump pays powerful tribute to ‘force of natur...,https://nypost.com/2026/02/17/us-news/trump-pa...,84
141,2026-02-17-13-26-00 +0000,nypost,"Iran’s supreme leader taunts Trump, US as high...",https://nypost.com/2026/02/17/us-news/irans-su...,79
27,2026-02-17-18-23-29 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...,76
148,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
195,91,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...
141,75,2026-02-17-13-26-00 +0000,nypost,"Iran’s supreme leader taunts Trump, US as high...",https://nypost.com/2026/02/17/us-news/irans-su...
148,58,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
97,51,2026-02-17-16-27-00 +0000,wsj,"Shooter Killed Ex-Wife, Son at Rhode Island Ho...",https://www.wsj.com/us-news/two-killed-three-i...
32,35,2026-02-17-18-14-06 +0000,nyt,Ukraine and Russia Hold New Round of Peace Tal...,https://www.nytimes.com/2026/02/17/world/europ...
256,31,2026-02-16-22-31-41 +0000,nypost,Former top Letitia James aide helps funnel mon...,https://nypost.com/2026/02/16/us-news/former-t...
208,30,2026-02-17-06-59-28 +0000,nyt,"Tom Pritzker, Hyatt Heir, Steps Down as Execut...",https://www.nytimes.com/2026/02/16/us/politics...
117,30,2026-02-17-15-13-46 +0000,nyt,"As Guthrie Case Grips Nation, One Police Chief...",https://www.nytimes.com/2026/02/16/us/nancy-gu...
158,28,2026-02-17-12-02-20 +0000,nypost,US citizen sentenced to four years in Russian ...,https://nypost.com/2026/02/17/world-news/us-ci...
121,27,2026-02-17-14-58-00 +0000,wsj,Eric Trump is investing in Israeli drone maker...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
